In [ ]:
pip install selenium

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
import pandas as pd
import time

In [ ]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

In [ ]:
URL_SEARCH = "https://www.amazon.com/Apple-AirPods-Charging-Latest-Model/product-reviews/B07PXGQC1Q/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber="
MAX_review = 1000
MAX_page = 100

In [ ]:
def scroll_to_bottom(browser): #Dùng để kéo xuống cuối trang
    # https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
    last_height = browser.execute_script('return document.body.scrollHeight')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(2)
        new_height = browser.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
def create_url_page_review(url):
    list_url_page_review = []
    for i in range(1, MAX_page + 1):
        temp = url + str(i)
        list_url_page_review.append(temp)
    return list_url_page_review

In [ ]:
list_url_page_review = create_url_page_review("https://www.amazon.com/Apple-AirPods-Charging-Latest-Model/product-reviews/B07PXGQC1Q/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=")

In [ ]:
list_url_page_review

['https://www.amazon.com/Apple-AirPods-Charging-Latest-Model/product-reviews/B07PXGQC1Q/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=1',
 'https://www.amazon.com/Apple-AirPods-Charging-Latest-Model/product-reviews/B07PXGQC1Q/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2',
 'https://www.amazon.com/Apple-AirPods-Charging-Latest-Model/product-reviews/B07PXGQC1Q/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=3',
 'https://www.amazon.com/Apple-AirPods-Charging-Latest-Model/product-reviews/B07PXGQC1Q/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=4',
 'https://www.amazon.com/Apple-AirPods-Charging-Latest-Model/product-reviews/B07PXGQC1Q/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=5',
 'https://www.amazon.com/Apple-AirPods-Charging-Latest-Model/product-reviews/B07PXGQC1Q/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_rev

In [ ]:
list_views = pd.DataFrame([],columns = ["name reviewer","review title","rating","review_body"])

with webdriver.Chrome(PATH) as browser:
    browser.maximize_window()
    for url in list_url_page_review:
        count = 0
        browser.get(url)
        time.sleep(1)
        arrOfViews = browser.find_elements(by=By.CLASS_NAME, value="a-section.celwidget")
        if count > MAX_review:
            break
        else:
            for i in range(1, len(arrOfViews)):
                review_body = arrOfViews[i].find_elements(by=By.CLASS_NAME, value="review-text-content")
                rating = arrOfViews[i].find_elements(by=By.CLASS_NAME, value="a-icon-alt")
                name_reviewer = arrOfViews[i].find_elements(by=By.CLASS_NAME, value="a-profile-name")
                review_title = arrOfViews[i].find_elements(by=By.CLASS_NAME, value="a-size-base.a-link-normal.review-title.a-color-base.review-title-content.a-text-bold")
                if len(review_body) > 0:
                    review = {"name reviewer": name_reviewer[0].text, "review title" : review_title[0].text,"rating": rating[0].get_attribute("innerText")[:3], "review_body": review_body[0].text}
                    list_views = list_views.append(review,ignore_index=True)
            count = count + len(arrOfViews)
        

<ipython-input-14-4b2c7827d84d>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(PATH) as browser:


In [ ]:
list_views

,name reviewer,review title,rating,review_body
0,Cam S.,Improved Over 1st Gen (regardless of what some...,5.0,Everyone is posting that there isn’t a differe...
2,Kevin scot,"Authentic and Beautiful, Love these way more t...",5.0,These AirPods are amazing they automatically p...
3,Atlas Heart,Don’t buy these!!,5.0,These are great but not much better then gen1....
4,Chih Y.,Defective...,1.0,I had the AirPods for only a few days.. every ...
5,margaret,MY SON LOVES THEM!!,5.0,My son really wanted airpods but his parents t...
...,...,...,...,...
1095,joanna,Case no longer charging,1.0,"Had it for only about 6-7 months, and the case..."
1096,Amazon User,These are awesome,5.0,I finally ditched my bose headphones since I f...
1097,Amazon Customer,Love these Irpods,5.0,I have a sudden hearing loss that was schedule...
1098,Kyle,Go with the original,5.0,I just came here to say I've tried 3 other bra...


In [ ]:
list_views = list_views.drop_duplicates(keep='last')
list_views = list_views.reset_index()
list_views = list_views.drop(["index"], axis=1)

In [ ]:
list_views

,name reviewer,review title,rating,review_body
0,Cam S.,Improved Over 1st Gen (regardless of what some...,5.0,Everyone is posting that there isn’t a differe...
1,Kevin scot,"Authentic and Beautiful, Love these way more t...",5.0,These AirPods are amazing they automatically p...
2,Atlas Heart,Don’t buy these!!,5.0,These are great but not much better then gen1....
3,Chih Y.,Defective...,1.0,I had the AirPods for only a few days.. every ...
4,margaret,MY SON LOVES THEM!!,5.0,My son really wanted airpods but his parents t...
...,...,...,...,...
995,joanna,Case no longer charging,1.0,"Had it for only about 6-7 months, and the case..."
996,Amazon User,These are awesome,5.0,I finally ditched my bose headphones since I f...
997,Amazon Customer,Love these Irpods,5.0,I have a sudden hearing loss that was schedule...
998,Kyle,Go with the original,5.0,I just came here to say I've tried 3 other bra...


In [ ]:
data = list_views
#split review body
data['single_review'] = data['review_body'].str.split('[.!?\n]+')
data = data.explode('single_review')

#drop columns that are not neccessary
data.drop(['name_reviewer', 'review_title', 'review_body'], axis = 1, inplace = True)

#reset index to remove MultiIndex
data.reset_index(drop = True, inplace = True)

In [ ]:
#remove empty value cells
data['single_review'].replace('', np.nan, inplace = True)
data.dropna(subset=['single_review'], inplace = True)

In [ ]:
#reset index
data.reset_index(drop = True, inplace = True)

In [ ]:
data.to_csv('Amazon_airpod_reviews.csv', sep ='\t')